<a href="https://colab.research.google.com/github/Tandon-A/SPAIC-Udacity/blob/master/SecuredFederatedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Secured Fedrated Learning**

Federated Learning allows for a machine learning model to train on user device's where the data is present. This allows to train models directly on the data without ever bringing the data to the company server. 
Once training is completed the models are brought upto the server for aggregation. 

As the model has learnt on user data, it holds information about the user data which can be used malicious users by using techniques such as checking the activation maps of the model for different set of inputs.

In order to prevent this, in secured federated learning, the aggregation of all the trained models is done by a trusted party and then this aggregated model is sent to the company server. 

In this example, the aggregation is performed by another virtual worker - 'arya'. 


In [0]:
!pip3 install syft 

In [2]:
import numpy as np
from torchvision import datasets, transforms
import torch.nn as nn
from torch import optim
from collections import OrderedDict
import torch 

#Loading the MNIST data
mnist = datasets.MNIST(root='./data', train=True, download=True, transform= transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
trainX = mnist.train_data
trainY = mnist.train_labels
np.random.seed(7)
#Shuffling MNIST data
perm = np.random.permutation(len(trainX))
trainX = trainX[perm,:]
trainY = trainY[perm]
#Preparing training data for two workers
val_x = trainX[40000:50000]
val_y = trainY[40000:50000]
tr_x1 = trainX[0:5000]
tr_y1 = trainY[0:5000]
tr_x2 = trainX[10000:15000]
tr_y2 = trainY[10000:15000]

0it [00:00, ?it/s]

9920512it [00:01, 8452691.77it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 126355.20it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2062542.87it/s]                            
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 47771.82it/s]            
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
#Function to create a NN model
def create_model():
  return nn.Sequential(OrderedDict([
    ('fc1',nn.Linear(784,512)),
    ('r1',nn.ReLU()),
    ('fc2',nn.Linear(512,256)),
    ('r2',nn.ReLU()),
    ('fc3',nn.Linear(256,64)),
    ('r3',nn.ReLU()),
    ('fc4',nn.Linear(64,10)),
    ('logps',nn.LogSoftmax(dim=1))
]))

criterion = nn.NLLLoss()

In [4]:
import syft as sy 

#Creating a hook between PySyft and Pytorch
hook = sy.TorchHook(torch)
#Creating three virtual workers
bob = sy.VirtualWorker(hook,"bob")
alice = sy.VirtualWorker(hook,"alice")
arya = sy.VirtualWorker(hook,"arya")

bob.clear_objects()
alice.clear_objects()
arya.clear_objects()
print (bob._objects,alice._objects,arya._objects)

{} {} {}


In [8]:
datasets = [(tr_x1,tr_y1),(tr_x2,tr_y2)]
models = list()
workers = [bob,alice]

for i in range(len(workers)):
  #sending validation data to specific worker
  val_x_worker = val_x.clone().send(workers[i])
  val_y_worker = val_y.clone().send(workers[i])
  tr_x,tr_y = datasets[i]
  #sending dataset to worker
  tr_x = tr_x.send(workers[i])
  tr_y = tr_y.send(workers[i])
  #creating a model for a worker
  model_worker = create_model()
  #creating optimizer for worker -- right now pysyft only allows for SGD as optimizer
  opt = optim.SGD(model_worker.parameters(),lr=0.001)
  model_worker.send(workers[i])
  
  
  #training worker models for 5 epochs 
  epochs = 5
  batch_size = 50
  batches = len(tr_x)//batch_size
  #setting worker model in train mode
  model_worker.train()
  print ("starting training for model ",i)
  for e in range(epochs):
    running_loss = 0
    for batch in range(batches):
      opt.zero_grad()
      images = tr_x[batch*batch_size : (batch+1)*batch_size].float().view(batch_size,-1)
      labels = tr_y[batch*batch_size : (batch+1)*batch_size]
      log_ps = model_worker(images)
      loss = criterion(log_ps,labels)
      running_loss += loss
      loss.backward()
      opt.step()
    running_loss = running_loss.get()
    #print (running_loss)
  #testing worker model on validation data  
  with torch.no_grad():
    model_worker.eval()
    ps = torch.exp(model_worker(val_x_worker.view(val_x_worker.shape[0],-1).float()))
    topp,topk = ps.topk(1,dim=1)
    equals = topk == val_y_worker.view(*topk.shape)
    equals = equals.get()
    accuracy = torch.mean(equals.type(torch.FloatTensor)) 
    print ("worker model = %r accuracy = %r " %(i,accuracy))
  #saving the worker model  
  models.append(model_worker)

starting training for model  0
worker model = 0 accuracy = tensor(0.9024) 
starting training for model  1
worker model = 1 accuracy = tensor(0.9015) 


In [0]:
model_alice = models[-1]
model_bob = models[-2]

print (model_alice.location,model_bob.location)
#Sending the models to arya worker
model_bob.move(arya)
model_alice.move(arya)

model = create_model()

#aggregating the models on arya worker device 
with torch.no_grad():
  for p1,p2,p3 in zip(model.parameters(),model_bob.parameters(),model_alice.parameters()):
    p1.set_((p2+p3).get()/2)  